In [126]:
import os
import sys
import numpy as np
from rdkit.Chem import MolFromSmiles
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook
import json
##### JSON modules #####
class NumpyEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, np.ndarray):
      return obj.tolist()
    return json.JSONEncoder.default(self, obj)
def save_json(data,filename):
  with open(filename, 'w') as fp:
    json.dump(data, fp, sort_keys=True, indent=4, cls=NumpyEncoder)
def load_json(filename):
  with open(filename, 'r') as fp:
    data = json.load(fp)
  return data
##### JSON modules #####

In [6]:
uniprot2triplets={}
with open('pfam_kinase_uniprot2triplets.txt','r') as f:
    for line in f:
        line=line.strip().split('\t')
        uniprot=line[0]
        triplets="[CLS] "+line[1] #add classifier token
        triplet_len=len(triplets.strip().split())
        while triplet_len<256:
            #pad sequence if short
            triplets=triplets+" [PAD]" #pad triplet sequence
            triplet_len+=1
        uniprot2triplets[uniprot]=triplets
    

In [25]:
with open('human_kinase_uniprot2triplets.json','r') as f:
    uniprot2triplets2=json.load(f)
for uni in uniprot2triplets2:
    if uni not in uniprot2triplets.keys():
        triplets="[CLS] "+uniprot2triplets2[uni]
        triplet_len=len(triplets.strip().split())
        while triplet_len<256:
            #pad sequence if short
            triplets=triplets+" [PAD]" #pad triplet sequence
            triplet_len+=1
        uniprot2triplets[uni]=triplets

In [54]:
save_json(uniprot2triplets,'uniprot2triplets.json')

In [7]:
ikey2smiles = {}
with open('/Users/hansaimlim/DrugTargetInteraction/data/Integrated/chemicals/integrated_chemicals.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        idx=int(line[0])
        ikey=line[1]
        smi=line[2]
        ikey2smiles[ikey]=smi

In [14]:
kinase_ligands=[]
kinases=[]
with open('/Users/hansaimlim/DrugTargetInteraction/data/Integrated/activities/tsv/integrated_kinases_continuous.tsv','r') as f:
    next(f)
    for line in f:
        line=line.strip().split('\t')
        ikey=line[1]
        uni=line[2]
        if ikey not in kinase_ligands:
            kinase_ligands.append(ikey)
        if uni not in kinases:
            kinases.append(uni)

In [110]:
kinases_available=[]
kinases_unavailable=[] #mostly non-human, uncommon proteins or mitochondrial proteins
ikeys_available=[]
ikeys_unavailable=[]
ikey2mol = {}
with open('inchikey_to_filter_out.txt','r') as f:
    for line in f: #inchikeys that cannot be processed by NeuMF
        line=line.strip()
        ikeys_unavailable.append(line)
        
for ikey in kinase_ligands:
    if ikey in ikeys_unavailable:
        continue
    try:
        mol = MolFromSmiles(ikey2smiles[ikey])
        ikey2mol[ikey]=mol
        ikeys_available.append(ikey)
    except:
        ikeys_unavailable.append(ikey) #inchikey that cannnot be processed by RDKit

for uni in kinases:
    if uni in uniprot2triplets.keys():
        kinases_available.append(uni)
    else:
        kinases_unavailable.append(uni)
print("{} ligands available; {} unavailable".format(len(ikeys_available),len(ikeys_unavailable)))
print("{} kinases available; {} unavailable".format(len(kinases_available),len(kinases_unavailable)))

119992 ligands available; 15147 unavailable
479 kinases available; 267 unavailable


In [127]:
len(ikeys_unavailable)

15147

In [112]:
pkipkd_train_pairs=[]
pkipkd_dev_pairs=[]
pkipkd_test_pairs=[]
pic50_train_pairs=[]
pic50_dev_pairs=[]
pic50_test_pairs=[]

with open('kinase_pki_pkd_train.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        pkipkd_train_pairs.append((ikey,uni,val))
with open('kinase_pki_pkd_dev.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        pkipkd_dev_pairs.append((ikey,uni,val))
with open('kinase_pki_pkd_test.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        pkipkd_test_pairs.append((ikey,uni,val))
with open('kinase_pic50_train.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        pic50_train_pairs.append((ikey,uni,val))
with open('kinase_pic50_dev.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        pic50_dev_pairs.append((ikey,uni,val))
with open('kinase_pic50_test.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        pic50_test_pairs.append((ikey,uni,val))

In [128]:
binary_train_pairs=[]
binary_dev_pairs=[]
binary_test_pairs=[]
with open('kinase_binary_train.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        binary_train_pairs.append((ikey,uni,int(val)))
with open('kinase_binary_dev.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        binary_dev_pairs.append((ikey,uni,int(val)))
with open('kinase_binary_test.tsv','r') as f:
    for line in f:
        line=line.strip().split('\t')
        ikey=line[0]
        uni=line[1]
        val=line[2]
        try:
            val=float(val)
        except:
            continue
        if ikey not in ikeys_available:
            continue
        if uni not in kinases_available:
            continue
        binary_test_pairs.append((ikey,uni,int(val)))
with open('kinase_binary_train.tsv.2','w') as out:
    for pair in binary_train_pairs:
        out.write("{}\t{}\t{}\n".format(pair[0],pair[1],pair[2]))
with open('kinase_binary_dev.tsv.2','w') as out:
    for pair in binary_dev_pairs:
        out.write("{}\t{}\t{}\n".format(pair[0],pair[1],pair[2]))
with open('kinase_binary_test.tsv.2','w') as out:
    for pair in binary_test_pairs:
        out.write("{}\t{}\t{}\n".format(pair[0],pair[1],pair[2]))

In [39]:
output_notebook()

Loading BokehJS ...

In [113]:
#pkipkd_train_pairs
#pkipkd_dev_pairs
#pkipkd_test_pairs

pkipkd_activities_by_kinase={kinase:[] for kinase in kinases_available}
for act in pkipkd_train_pairs:
    pkipkd_activities_by_kinase[act[1]].append(act)

hist, edges = np.histogram([act[2] for act in pkipkd_train_pairs], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd training activities',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

/anaconda3/envs/rdkit/lib/python3.6/site-packages/bokeh/models/plots.py:756: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
/anaconda3/envs/rdkit/lib/python3.6/site-packages/bokeh/models/plots.py:756: UserWarning: 
You are attempting to set `plot.legend.background_fill_color` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


In [116]:
pkipkd_train_filtered=[]
#pkipkd_activities_by_kinase={kinase:[] for kinase in kinases_available}
for uni in pkipkd_activities_by_kinase.keys():
    activity_near_4=[] #used to downsample activity=4.17
    activity_near_5=[] #used to downsample activity=5.0
    activity_at_417=[] #activities between 4.17 and 4.18
    activity_at_5=[] #activities at 5.0
    rest=[] #other activities
    for act in pkipkd_activities_by_kinase[uni]:
        if (act[2]>=4.0) and (act[2]<4.1):
            activity_near_4.append(act)
        elif (act[2]>=4.2) and (act[2]<4.3):
            activity_near_4.append(act)
        elif (act[2]>=4.9) and (act[2]<5.0):
            activity_near_5.append(act)
        elif (act[2]>=5.1) and (act[2]<5.2):
            activity_near_5.append(act)
        elif (act[2]>4.176) and (act[2]<4.18):
            activity_at_417.append(act)
        elif (act[2]==5.0):
            activity_at_5.append(act)
        else:
            rest.append(act)
    max_near_4=int(np.ceil((len(activity_near_4)/2)*1.0))
    max_near_5=int(np.ceil((len(activity_near_5)/2)*1.0))
    
    if len(activity_at_417)>max_near_4:
        np.random.shuffle(activity_at_417)
        activity_at_417=activity_at_417[:max_near_4] #discard rest of activities at 4.176
    if len(activity_at_5)>max_near_5:
        np.random.shuffle(activity_at_5)
        activity_at_5=activity_at_5[:max_near_5] #discard rest of activities at 5.0
        
    activities_chosen=activity_near_4+activity_near_5+rest+activity_at_417+activity_at_5
    pkipkd_train_filtered+=activities_chosen
print("After filtering activities at 4.17 and 5.0, {} activities selected for pKi-pKd training".format(len(pkipkd_train_filtered)))


hist, edges = np.histogram([act[2] for act in pkipkd_train_filtered], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd training activities after filtering',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

After filtering activities at 4.17 and 5.0, 526416 activities selected for pKi-pKd training


In [79]:
np.log(5.0)

1.6094379124341003

In [102]:
np.exp(2.97)

19.49191959603112

In [122]:
hist, edges = np.histogram([act[2] for act in pkipkd_train_filtered], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd training activities after filtering',tools='box_zoom,save,reset', width=550,
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"

hist, edges = np.histogram([np.log(act[2]) for act in pkipkd_train_filtered], density=False, range=(1,np.log(14)),bins=80)
p1 = figure(title='x-axis log transformed',tools='box_zoom,save,reset', width=550,
           background_fill_color="#fafafa",x_range=(1,np.log(14)))
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p1.y_range.start = 0
p1.legend.location = "center_right"
p1.legend.background_fill_color = "#fefefe"
show(gridplot([[p,p1]]))

/anaconda3/envs/rdkit/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in log
  # Remove the CWD from sys.path while we load stuff.


In [123]:
#Filter dev pairs
pkipkd_dev_activities_by_kinase={kinase:[] for kinase in kinases_available}
for act in pkipkd_dev_pairs:
    pkipkd_dev_activities_by_kinase[act[1]].append(act)

hist, edges = np.histogram([act[2] for act in pkipkd_dev_pairs], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd Dev activities',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

pkipkd_dev_filtered=[]
#pkipkd_activities_by_kinase={kinase:[] for kinase in kinases_available}
for uni in pkipkd_dev_activities_by_kinase.keys():
    activity_near_4=[] #used to downsample activity=4.17
    activity_near_5=[] #used to downsample activity=5.0
    activity_at_417=[] #activities between 4.17 and 4.18
    activity_at_5=[] #activities at 5.0
    rest=[] #other activities
    for act in pkipkd_dev_activities_by_kinase[uni]:
        if (act[2]>=4.0) and (act[2]<4.1):
            activity_near_4.append(act)
        elif (act[2]>=4.2) and (act[2]<4.3):
            activity_near_4.append(act)
        elif (act[2]>=4.9) and (act[2]<5.0):
            activity_near_5.append(act)
        elif (act[2]>=5.1) and (act[2]<5.2):
            activity_near_5.append(act)
        elif (act[2]>4.176) and (act[2]<4.18):
            activity_at_417.append(act)
        elif (act[2]==5.0):
            activity_at_5.append(act)
        else:
            rest.append(act)
    max_near_4=int(np.ceil((len(activity_near_4)/2)*1.0))
    max_near_5=int(np.ceil((len(activity_near_5)/2)*1.0))
    
    if len(activity_at_417)>max_near_4:
        np.random.shuffle(activity_at_417)
        activity_at_417=activity_at_417[:max_near_4] #discard rest of activities at 4.176
    if len(activity_at_5)>max_near_5:
        np.random.shuffle(activity_at_5)
        activity_at_5=activity_at_5[:max_near_5] #discard rest of activities at 5.0
        
    activities_chosen=activity_near_4+activity_near_5+rest+activity_at_417+activity_at_5
    pkipkd_dev_filtered+=activities_chosen
print("After filtering activities at 4.17 and 5.0, {} activities selected for pKi-pKd dev".format(len(pkipkd_dev_filtered)))


hist, edges = np.histogram([act[2] for act in pkipkd_dev_filtered], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd dev activities after filtering',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

After filtering activities at 4.17 and 5.0, 10566 activities selected for pKi-pKd dev


In [124]:
#Filter test pairs
pkipkd_test_activities_by_kinase={kinase:[] for kinase in kinases_available}
for act in pkipkd_test_pairs:
    pkipkd_test_activities_by_kinase[act[1]].append(act)

hist, edges = np.histogram([act[2] for act in pkipkd_test_pairs], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd test activities',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

pkipkd_test_filtered=[]
#pkipkd_activities_by_kinase={kinase:[] for kinase in kinases_available}
for uni in pkipkd_test_activities_by_kinase.keys():
    activity_near_4=[] #used to downsample activity=4.17
    activity_near_5=[] #used to downsample activity=5.0
    activity_at_417=[] #activities between 4.17 and 4.18
    activity_at_5=[] #activities at 5.0
    rest=[] #other activities
    for act in pkipkd_test_activities_by_kinase[uni]:
        if (act[2]>=4.0) and (act[2]<4.1):
            activity_near_4.append(act)
        elif (act[2]>=4.2) and (act[2]<4.3):
            activity_near_4.append(act)
        elif (act[2]>=4.9) and (act[2]<5.0):
            activity_near_5.append(act)
        elif (act[2]>=5.1) and (act[2]<5.2):
            activity_near_5.append(act)
        elif (act[2]>4.176) and (act[2]<4.18):
            activity_at_417.append(act)
        elif (act[2]==5.0):
            activity_at_5.append(act)
        else:
            rest.append(act)
    max_near_4=int(np.ceil((len(activity_near_4)/2)*1.0))
    max_near_5=int(np.ceil((len(activity_near_5)/2)*1.0))
    
    if len(activity_at_417)>max_near_4:
        np.random.shuffle(activity_at_417)
        activity_at_417=activity_at_417[:max_near_4] #discard rest of activities at 4.176
    if len(activity_at_5)>max_near_5:
        np.random.shuffle(activity_at_5)
        activity_at_5=activity_at_5[:max_near_5] #discard rest of activities at 5.0
        
    activities_chosen=activity_near_4+activity_near_5+rest+activity_at_417+activity_at_5
    pkipkd_test_filtered+=activities_chosen
print("After filtering activities at 4.17 and 5.0, {} activities selected for pKi-pKd test set".format(len(pkipkd_test_filtered)))


hist, edges = np.histogram([act[2] for act in pkipkd_test_filtered], density=False, range=(0,15),bins=150)
p = figure(title='pKi,pKd test activities after filtering',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,15))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

After filtering activities at 4.17 and 5.0, 2696 activities selected for pKi-pKd test set


In [125]:
with open('kinase_pki_pkd_train_adj.tsv','w') as out:
    for act in pkipkd_train_filtered:
        out.write("{}\t{}\t{}\n".format(act[0],act[1],act[2]))
with open('kinase_pki_pkd_dev_adj.tsv','w') as out:
    for act in pkipkd_dev_filtered:
        out.write("{}\t{}\t{}\n".format(act[0],act[1],act[2]))
with open('kinase_pki_pkd_test_adj.tsv','w') as out:
    for act in pkipkd_test_filtered:
        out.write("{}\t{}\t{}\n".format(act[0],act[1],act[2]))

In [91]:
pic50_train_activities_by_kinase={kinase:[] for kinase in kinases_available}
for act in pic50_train_pairs:
    pic50_train_activities_by_kinase[act[1]].append(act)

hist, edges = np.histogram([act[2] for act in pic50_train_pairs], density=False, range=(0,14.5),bins=100)
p = figure(title='pIC50 training activities',tools='box_zoom,save,reset', 
           background_fill_color="#fafafa",x_range=(0,14.5))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)
p.y_range.start = 0
p.legend.location = "center_right"
p.legend.background_fill_color = "#fefefe"
show(p)

In [55]:
import torch

labels = torch.tensor([1, 2, 3, 5])
one_hot = torch.zeros(4, 6)
one_hot[torch.arange(4), labels] = 1

In [64]:
files=sorted(os.listdir('./'))
ikeys_to_filter=[]
with open(files[3],'r') as f:
    for line in f:
        line=line.strip()
        ikeys_to_filter.append(line)
len(ikeys_to_filter)

114

In [92]:
files[5]

'kinase_binary_test.tsv'

In [67]:
files[5]+'.filt'

'kinase_binary_test.tsv.filt'

In [68]:
for inf in files[4:-1]:
    with open(inf+'.filt','w') as out:
        with open(inf, 'r') as f:
            for line in f:
                acts=line.strip().split('\t')
                if acts[0] not in ikeys_to_filter:
                    out.write(line)
                

In [72]:
batch_labels=[0,1,1,0,1]
one_hot = torch.zeros(5, 2)
one_hot[torch.arange(5), batch_labels] = 1
one_hot.long()

tensor([[1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1]])

In [109]:
-np.log10(50e-6)

4.301029995663981